In [1]:
%%capture
!pip install import_ipynb --no-cache
import import_ipynb
m = __import__("Methods")

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from urllib.request import urlretrieve
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy.stats import boxcox
import pandas as pd
import zipfile
import urllib.request
import os

In [3]:
rna_seq = dict()
phenotype = dict()
meta = dict()

In [ ]:
rna_seq["47"] = pd.read_csv("../raw_data/GLDS-47_rna_seq_Normalized_Counts.csv")
rna_seq["48"] = pd.read_csv("../raw_data/GLDS-48_rna_seq_Normalized_Counts.csv")
rna_seq["137"] = pd.read_csv("../raw_data/GLDS-137_rna_seq_Normalized_Counts.csv")
rna_seq["168"] = pd.read_csv("../raw_data/GLDS-168_rna_seq_Normalized_Counts.csv")

In [15]:
def read_meta_data(dataset):
  # dataset=255
  url = 'https://osdr.nasa.gov/geode-py/ws/studies/OSD-' + str(dataset) + '/download?source=datamanager&file=OSD-' + dataset + '_metadata_OSD-' + dataset + '-ISA.zip'
  filename = dataset + '-meta.zip'
  urlretrieve(url, filename)
  !unzip -o {filename} > /dev/null
  df = pd.read_csv('s_OSD-' + dataset + '.txt', sep='\t', header=0)
  return df

In [5]:
def fill_na_with_group_avg(df):
    # Extract the group from the 'source_name' column
    df['group'] = df['source_name'].apply(lambda x: x.split('_')[0])
    
    # For each column that needs NaN values filled
    for col in ['th_positive_cells', 'repo_glial_cells']:
        # Calculate the group average for each group
        group_avg = df.groupby('group')[col].transform('mean')
        
        # Fill NaN values with the group average
        df[col] = df[col].fillna(group_avg)
    
    # Drop the temporary 'group' column
    df.drop(columns=['group'], inplace=True)